In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## चरण 1: संरचित आउटपुटका लागि Pydantic मोडेलहरू परिभाषित गर्नुहोस्

यी मोडेलहरूले एजेन्टहरूले फर्काउने **schema** परिभाषित गर्छन्। Pydantic सँग `response_format` प्रयोग गर्दा सुनिश्चित हुन्छ:
- ✅ प्रकार-सुरक्षित डाटा निकासी
- ✅ स्वचालित प्रमाणीकरण
- ✅ स्वतन्त्र-पाठ प्रतिक्रियाहरूबाट कुनै पार्सिङ त्रुटि हुँदैन
- ✅ फिल्डहरूमा आधारित सजिलो सशर्त मार्गनिर्देशन


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## चरण २: होटल बुकिङ उपकरण सिर्जना गर्नुहोस्

यो उपकरण **availability_agent** ले कोठाहरू उपलब्ध छन् कि छैनन् जाँच गर्न कल गर्नेछ। हामी `@ai_function` डेकोरेटर प्रयोग गर्छौं:
- पाइथन फंक्शनलाई AI-कल गर्न मिल्ने उपकरणमा रूपान्तरण गर्न
- LLM का लागि JSON स्किमा स्वचालित रूपमा उत्पन्न गर्न
- प्यारामीटर मान्यकरण सम्हाल्न
- एजेन्टहरूले स्वचालित रूपमा कल गर्न सक्ने बनाउन

यस डेमोको लागि:
- **स्टकहोम, सिएटल, टोक्यो, लन्डन, एम्स्टर्डम** → कोठाहरू उपलब्ध छन् ✅
- **अन्य सबै शहरहरू** → कोठाहरू उपलब्ध छैनन् ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## चरण ३: मार्गनिर्देशनका लागि शर्त कार्यहरू परिभाषित गर्नुहोस्

यी कार्यहरूले एजेन्टको प्रतिक्रियाको निरीक्षण गर्दछन् र कार्यप्रवाहमा कुन मार्ग लिनुपर्ने निर्णय गर्दछन्।

**प्रमुख ढाँचा:**
1. सन्देश `AgentExecutorResponse` हो कि होईन जाँच गर्नुहोस्
2. संरचित आउटपुट (Pydantic मोडेल) पार्स गर्नुहोस्
3. रुटिङ नियन्त्रण गर्न `True` वा `False` फर्काउनुहोस्

कार्यप्रवाहले कुन कार्यान्वयनकर्ता अर्को बोलाउने निर्धारण गर्न यी शर्तहरू **एजहरू** मा मूल्याङ्कन गर्नेछ।


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Step 4: अनुकूलित प्रदर्शन कार्यान्वयनकर्ता बनाउनुहोस्

कार्यान्वयनकर्ताहरू workflow का यस्ता कम्पोनेन्टहरू हुन् जुन रूपान्तरण वा साइड इफेक्टहरू गर्दछ। हामीले अन्तिम परिणाम प्रदर्शन गर्ने अनुकूलित कार्यान्वयनकर्ता बनाउन `@executor` डेकोरेटर प्रयोग गर्छौं।

**मुख्य परिकल्पना:**
- `@executor(id="...")` - एक कार्यक्षेत्र कार्यान्वयनकर्ता रूपमा फंक्शन दर्ता गर्दछ
- `WorkflowContext[Never, str]` - इनपुट/आउटपुट को लागि प्रकार संकेतहरू
- `ctx.yield_output(...)` - अन्तिम workflow परिणाम दिन्छ


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Step 5: वातावरण भेरिएबलहरू लोड गर्नुहोस्

LLM क्लाइन्ट कन्फिगर गर्नुहोस्। यो उदाहरणसँग काम गर्दछ:
- **GitHub मोडेलहरू** (GitHub टोकन सहित निःशुल्क तह)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Step 6: संरचित आउटपुट सहित AI एजेन्टहरू सिर्जना गर्नुहोस्

हामी **तीन विशेष एजेन्टहरू** सिर्जना गर्छौं, प्रत्येक `AgentExecutor` मा राखिएका छन्:

1. **availability_agent** - उपकरण प्रयोग गरी होटल उपलब्धता जाँच गर्छ
2. **alternative_agent** - वैकल्पिक शहरहरू सुझाव दिन्छ (जब कोठा नहुन)
3. **booking_agent** - बुकिङ प्रोत्साहित गर्छ (जब कोठा उपलब्ध हुन्छ)

**महत्त्वपूर्ण विशेषताहरू:**
- `tools=[hotel_booking]` - एजेन्टलाई उपकरण प्रदान गर्छ
- `response_format=PydanticModel` - संरचित JSON आउटपुटलाई अनिवार्य बनाउँछ
- `AgentExecutor(..., id="...")` - कार्यप्रवाह प्रयोगका लागि एजेन्टलाई ल्याप्छ


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Step 7: सशर्त एजहरूको साथ वर्कफ्लो निर्माण गर्नुहोस्

अब हामी `WorkflowBuilder` प्रयोग गरेर सशर्त राउटिङ सहित ग्राफ निर्माण गर्छौं:

**वर्कफ्लो संरचना:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**मुख्य मेथडहरू:**
- `.set_start_executor(...)` - प्रवेश बिन्दु सेट गर्छ
- `.add_edge(from, to, condition=...)` - सशर्त एज थप्छ
- `.build()` - वर्कफ्लोलाई अन्तिम रूप दिन्छ


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Step 8: Run Test Case 1 - शहर बिना उपलब्धता (पेरिस)

हाम्रो सिमुलेशनमा कुनै कोठा नभएको पेरिसमा होटलहरू अनुरोध गरेर **कोई उपलब्धता नभएको** बाटो परीक्षण गरौं।


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Step 9: टेस्ट केस 2 चलाउनुहोस् - शहर WITH उपलब्धता (स्टकहोम)

अब हामी **उपलब्धता** पथलाई परीक्षण गर्नेछौं स्टकहोममा होटलहरू अनुरोध गरेर (जहाँ हाम्रो सिमुलेशनमा कोठाहरू छन्)।


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## मुख्य निष्कर्ष र आगामी कदमहरू

### ✅ तपाईंले सिक्नुभएको कुरा:

1. **WorkflowBuilder ढाँचा**
   - प्रवेश बिन्दु परिभाषित गर्न `.set_start_executor()` प्रयोग गर्नुहोस्
   - सर्त अनुसार मार्गनिर्देशनका लागि `.add_edge(from, to, condition=...)` प्रयोग गर्नुहोस्
   - कार्यप्रवाहलाई अन्तिम रूप दिन `.build()` कल गर्नुहोस्

2. **सर्तीय मार्गनिर्देशन**
   - सर्तका कार्यहरूले `AgentExecutorResponse` जाँच गर्छन्
   - मार्गनिर्देशन निर्णय गर्न संरचित आउटपुटहरू पार्स गर्छन्
   - कुनै छेउ सक्रिय गर्न `True` र छोड्न `False` फर्काउँछन्

3. **उपकरण एकीकरण**
   - Python कार्यहरूलाई AI उपकरणमा रूपान्तरण गर्न `@ai_function` प्रयोग गर्नुहोस्
   - आवश्यक पर्दा एजेन्टहरूले उपकरणहरू स्वचालित रूपमा कल गर्छन्
   - उपकरणहरूले JSON फर्काउँछन् जुन एजेन्टहरूले पार्स गर्न सक्छन्

4. **संरचित आउटपुटहरू**
   - प्रकार-सुरक्षित डाटा निक्षेपणका लागि Pydantic मोडेलहरू प्रयोग गर्नुहोस्
   - एजेन्टहरू सिर्जना गर्दा `response_format=MyModel` सेट गर्नुहोस्
   - प्रतिक्रियाहरू `Model.model_validate_json()` प्रयोग गरी पार्स गर्नुहोस्

5. **कस्टम एक्जिक्युटरहरू**
   - कार्यप्रवाहका कम्पोनेन्टहरू बनाउन `@executor(id="...")` प्रयोग गर्नुहोस्
   - एक्जिक्युटरहरूले डाटा रूपान्तरण गर्न वा साइड इफेक्टहरू गर्न सक्छन्
   - कार्यप्रवाह परिणाम उत्पादन गर्न `ctx.yield_output()` प्रयोग गर्नुहोस्

### 🚀 वास्तविक-विश्व अनुप्रयोगहरू:

- **यात्रा बुकिंग**: उपलब्धता जाँच, विकल्पहरू सुझाव, विकल्पहरूको तुलना
- **ग्राहक सेवा**: समस्या प्रकार, भावना, प्राथमिकतामा आधारित मार्गनिर्देशन
- **ई-वाणिज्य**: इनभेन्टरी जाँच, विकल्पहरू सुझाव, अर्डर प्रक्रिया
- **सामग्री नियमन**: विषाक्तता स्कोर, प्रयोगकर्ता झण्डाहरूको आधारमा मार्गनिर्देशन
- **अनुमोदन कार्यप्रवाहहरू**: रकम, प्रयोगकर्ता भूमिका, जोखिम स्तरमा आधारित मार्गनिर्देशन
- **बहु-चरण प्रशोधन**: डाटा गुणस्तर, पूर्णतामा आधारित मार्गनिर्देशन

### 📚 आगामी कदमहरू:

- थप जटिल सर्तहरू थप्ने (एक भन्दा बढी मापदण्डहरू)
- कार्यप्रवाह स्थिति व्यवस्थापन सहित लूपहरू कार्यान्वयन गर्ने
- पुन: प्रयोग गर्न मिल्ने कम्पोनेन्टहरूका लागि उप-कार्यप्रवाहहरू थप्ने
- वास्तविक API हरूसँग एकीकरण गर्ने (होटल बुकिंग, इनभेन्टरी प्रणालीहरू)
- त्रुटि ह्यान्डलिङ र फ्यालब्याक मार्गहरू थप्ने
- बिल्ट-इन भिजुअलाइजेसन उपकरणहरूसँग कार्यप्रवाहहरू दृश्यात्मक बनाउने


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:  
यस दस्तावेजलाई AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) को प्रयोग गरी अनुवाद गरिएको हो। हामी सटीकता सुनिश्चित गर्न प्रयासरत छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अनिश्चितताहरू हुन सक्छन्। मूल दस्तावेज यसको मातृभाषामा आधिकारिक स्रोतको रुपमा मानिनु पर्छ। महत्वपूर्ण जानकारीको लागि व्यावसायिक मानवीय अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न कुनै पनि गलत बुझाइ वा गलत व्याख्याका लागि हामी जिम्मेवार छैनौं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
